## Load Data Set

In [3]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)


root_path = './'
dataset_path = os.path.join(root_path, 'tf_dataset_gray')


models_path = os.path.join(root_path, 'saved_models_face_v9')
if not os.path.exists(models_path):
    os.mkdir(models_path)


BATCH_SIZE = 256


image_feature_description = {
    'enc': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    'img': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True)
}


def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)


raw_train_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'train.tfrecords'))
parsed_train_dataset = raw_train_dataset.map(_parse_image_function)

# raw_val_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'val.tfrecords'))
# parsed_val_dataset = raw_val_dataset.map(_parse_image_function)

# raw_test_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'test.tfrecords'))
# parsed_test_dataset = raw_test_dataset.map(_parse_image_function)


parsed_train_dataset = parsed_train_dataset.repeat()
parsed_train_dataset = parsed_train_dataset.shuffle(7000)
parsed_train_dataset = parsed_train_dataset.batch(BATCH_SIZE)
dataset_iterator = parsed_train_dataset.make_one_shot_iterator()

variable_dataset = dataset_iterator.get_next()

1.13.1


## Model Design

In [22]:
enc_len = 0
age_len = 8
img_shape = (32, 32, 1)
width, height, depth = (32, 32, 1)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 50  # 32 x 32 x 3
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len


def build_discriminator():
    conv = keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=16,
            kernel_size=(3, 3),
            strides=2,
            input_shape=img_shape
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),

        # conv block 2
        keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=3,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
    ])
    
    print("D M1:")
    conv.summary()
    
    model = keras.Sequential([
        # output
        keras.layers.Dense(1, input_shape=(age_len+27,)),
        keras.layers.Activation(tf.nn.sigmoid),
    ])
    
    clf = keras.Sequential([
        # output
        keras.layers.Dense(age_len, input_shape=(age_len+27,)),
        keras.layers.Activation(tf.nn.softmax),
    ])
    
    print("D M2:")
    model.summary()
    
    # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # convolution
    zout = conv(z)
    
    # flatten image
    z_flat = keras.layers.Flatten()(zout)
    
    # concatenation
    inputs = keras.layers.concatenate([c2, z_flat])
    
    # real or fake
    outputs = model(inputs)
    
    # 0, 1, 2, 3, .. 9
    lbl = clf(inputs)
    
    return keras.models.Model([c2, z], [outputs, lbl])


def build_generator():
    
    conv = keras.Sequential([
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=16,
            kernel_size=(3, 3),
            strides=1,
            input_shape=(12, 12, 1)
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=18,
            kernel_size=(3, 3),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=20,
            kernel_size=(3, 3),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=22,
            kernel_size=(3, 3),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 5
        keras.layers.Conv2DTranspose(
            filters=24,
            kernel_size=(3, 3),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 6
        keras.layers.Conv2DTranspose(
            filters=16,
            kernel_size=(3, 3),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 7
        keras.layers.Conv2DTranspose(
            filters=8,
            kernel_size=(3, 3),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 8
        keras.layers.Conv2DTranspose(
            filters=4,
            kernel_size=(3, 3),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 9
        keras.layers.Conv2DTranspose(
            filters=2,
            kernel_size=(3, 3),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),

        # transpose conv block 10
        keras.layers.Conv2DTranspose(
            filters=1,
            kernel_size=(3, 3),
            strides=1
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])
    
    print("G M2:")
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(144, input_shape=(input_dim,)),
        keras.layers.Activation(tf.nn.relu),
        
        # reshape 1d to 3d
        keras.layers.Reshape((12, 12, 1))
    ])
    
    print("G M1:")
    model.summary()
    
    # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))

    # concatenation
    inputs = keras.layers.concatenate([c2, x])
    
    # flat dense output
    out_1 = model(inputs)
    
    # transpose conv output
    outputs = conv(out_1)
    
    return keras.models.Model([c2, x], outputs)


discriminator = build_discriminator()
generator = build_generator()

D M1:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 15, 15, 16)        160       
_________________________________________________________________
activation_192 (Activation)  (None, 15, 15, 16)        0         
_________________________________________________________________
batch_normalization_107 (Bat (None, 15, 15, 16)        64        
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 7, 7, 32)          4640      
_________________________________________________________________
activation_193 (Activation)  (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_108 (Bat (None, 7, 7, 32)          128       
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 3, 3, 3)           867       
____

In [23]:
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_71 (InputLayer)           (None, 8)            0                                            
__________________________________________________________________________________________________
input_72 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
concatenate_36 (Concatenate)    (None, 58)           0           input_71[0][0]                   
                                                                 input_72[0][0]                   
__________________________________________________________________________________________________
sequential_90 (Sequential)      (None, 12, 12, 1)    8496        concatenate_36[0][0]             
__________

In [24]:
discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_70 (InputLayer)           (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
sequential_86 (Sequential)      (None, 3, 3, 3)      5859        input_70[0][0]                   
__________________________________________________________________________________________________
input_69 (InputLayer)           (None, 8)            0                                            
__________________________________________________________________________________________________
flatten_18 (Flatten)            (None, 27)           0           sequential_86[1][0]              
__________________________________________________________________________________________________
concatenat

## Generative Adversarial Network

In [25]:
GLR = 0.0009  # generator
DLR = 0.0009  # discriminator


# Wasserstein
def d_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=[keras.losses.binary_crossentropy, keras.losses.categorical_crossentropy],
    metrics=['accuracy']
)


# condition
# c1 = keras.layers.Input(shape=(enc_len,))
c2 = keras.layers.Input(shape=(age_len,))

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator([c2, x])
out, lbl = discriminator([c2, z])

# GAN
gan = keras.models.Model(inputs=[c2, x], outputs=[out, lbl])

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=[keras.losses.binary_crossentropy, keras.losses.categorical_crossentropy],
    metrics=['accuracy'])

In [26]:
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_73 (InputLayer)           (None, 8)            0                                            
__________________________________________________________________________________________________
input_74 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
model_36 (Model)                (None, 32, 32, 1)    28821       input_73[0][0]                   
                                                                 input_74[0][0]                   
__________________________________________________________________________________________________
model_35 (Model)                [(None, 1), (None, 8 6183        input_73[0][0]                   
          

## Visualization Method

In [27]:
# from google.colab import drive
import os


# drive.mount('/content/gdrive', force_remount=True)

root_path = './'
tgt_pth = os.path.join(root_path, 'visualize_face-v23')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

In [36]:
def visualizeGAN(e, z_real, z_fake, conditions=None):

    fig, axes = plt.subplots(2, 4, figsize=(20, 10))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0 and z_real is not None:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_real[r_real * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )
            else:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_fake[r_fake * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )
                cell.set_title(
                    str(
                        np.argmax(
                            conditions[r_fake * 4 + col]
                        )
                    ) + ": " + str(
                        conditions[r_fake * 4 + col]
                    )
                )

            cell.axis("off")

        if row % 2 == 0 and z_real is not None:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

## Load Batch

In [37]:
def load_noise():
    
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_true = y_true.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            # concatenate face + age + noise
#             c1 = row['enc']
            c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            
            yield c2, x, y_true


def load_batch():
    
    y_fake = tf.zeros((BATCH_SIZE,))
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_fake = y_fake.eval()
        y_true = y_true.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            z_fake = generator.predict([c2, x])

            # real data
            z_real = tf.reshape(tf.io.decode_raw(row['img'], tf.int64), (-1, width, height, depth))
    
            z_real = tf.cast(z_real, tf.float32)
    
            # scale to [-1, +1]
            z_real = tf.math.subtract(tf.math.divide(z_real, 127.5), 1)
        
            z_real = z_real.eval()
                        
            yield c2, x, z_fake, y_fake, z_real, y_true

## Train Model

## Train Model

In [ ]:
EPOCHS = 8000
STEPS = 1  # 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


disc_itr = load_batch()
gen_itr = load_noise()


# fixed test sample
noise = np.random.normal(size=(age_len, noise_len,) )
conditions = np.identity(age_len)


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS):
        
        c2, x, z_fake, y_fake, z_real, y_real = next(disc_itr)
    
        # train
        ## ['loss', 'model_1_loss', 'model_1_loss', 'model_1_acc', 'model_1_acc_1']
        loss_1, _, _, acc_1, _ = discriminator.train_on_batch([c2, z_real], [y_real, c2])
        loss_2, _, _, acc_2, _ = discriminator.train_on_batch([c2, z_fake], [y_fake, c2])

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

      c2, x, y_true = next(gen_itr)

      # train
      ## ['loss', 'model_1_loss', 'model_1_loss', 'model_1_acc', 'model_1_acc_1']
      loss_1, _, _, acc_1, _ = gan.train_on_batch([c2, x], [y_true, c2])

      loss.append(loss_1)
      acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, D:[ACC: %{:.2f}, LOSS: {:.2f}], G:[ACC: %{:.2f}, LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 100 == 0:
        ## visualize results
        synthesized = generator.predict([conditions, noise])
        visualizeGAN(e, None, synthesized, conditions)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Exception ignored in: <generator object load_batch at 0x7fac5de82830>
Traceback (most recent call last):
  File "<ipython-input-34-134ecb5784d7>", line 77, in load_batch
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1573, in __exit__
    exec_tb)
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5069, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.client.session.Session'> objects
Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7fac5de99208>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "/usr/

E: 0, D:[ACC: %55.66, LOSS: 270.03], G:[ACC: %22.27, LOSS: 380.58]
E: 1, D:[ACC: %57.62, LOSS: 286.27], G:[ACC: %17.97, LOSS: 350.56]
E: 2, D:[ACC: %56.05, LOSS: 268.32], G:[ACC: %16.80, LOSS: 319.05]
E: 3, D:[ACC: %61.52, LOSS: 264.28], G:[ACC: %12.50, LOSS: 317.68]
E: 4, D:[ACC: %70.51, LOSS: 256.27], G:[ACC: %10.55, LOSS: 332.53]
E: 5, D:[ACC: %64.45, LOSS: 257.98], G:[ACC: %5.86, LOSS: 335.25]
E: 6, D:[ACC: %72.85, LOSS: 246.10], G:[ACC: %3.52, LOSS: 340.41]
E: 7, D:[ACC: %77.73, LOSS: 249.35], G:[ACC: %5.08, LOSS: 327.60]
E: 8, D:[ACC: %70.51, LOSS: 247.00], G:[ACC: %7.03, LOSS: 320.10]
E: 9, D:[ACC: %75.20, LOSS: 242.32], G:[ACC: %3.91, LOSS: 350.69]
E: 10, D:[ACC: %76.17, LOSS: 236.38], G:[ACC: %1.17, LOSS: 369.24]
E: 11, D:[ACC: %77.93, LOSS: 229.85], G:[ACC: %0.00, LOSS: 398.55]
E: 12, D:[ACC: %79.88, LOSS: 225.45], G:[ACC: %0.00, LOSS: 386.73]
E: 13, D:[ACC: %91.99, LOSS: 230.25], G:[ACC: %0.00, LOSS: 386.66]
E: 14, D:[ACC: %89.45, LOSS: 227.99], G:[ACC: %0.39, LOSS: 389.10]


E: 122, D:[ACC: %99.22, LOSS: 165.92], G:[ACC: %0.39, LOSS: 406.05]
E: 123, D:[ACC: %99.41, LOSS: 163.92], G:[ACC: %0.78, LOSS: 347.43]
E: 124, D:[ACC: %98.24, LOSS: 165.46], G:[ACC: %3.91, LOSS: 323.75]
E: 125, D:[ACC: %100.00, LOSS: 157.30], G:[ACC: %0.39, LOSS: 332.52]
E: 126, D:[ACC: %100.00, LOSS: 158.45], G:[ACC: %4.69, LOSS: 314.14]
E: 127, D:[ACC: %99.80, LOSS: 162.90], G:[ACC: %5.47, LOSS: 286.86]
E: 128, D:[ACC: %99.61, LOSS: 158.84], G:[ACC: %10.55, LOSS: 274.32]
E: 129, D:[ACC: %99.80, LOSS: 157.26], G:[ACC: %22.66, LOSS: 253.20]
E: 130, D:[ACC: %99.80, LOSS: 152.55], G:[ACC: %26.56, LOSS: 239.20]
E: 131, D:[ACC: %99.80, LOSS: 154.07], G:[ACC: %33.59, LOSS: 233.79]
E: 132, D:[ACC: %99.80, LOSS: 153.67], G:[ACC: %28.52, LOSS: 236.13]
E: 133, D:[ACC: %100.00, LOSS: 154.03], G:[ACC: %27.34, LOSS: 244.84]
E: 134, D:[ACC: %100.00, LOSS: 149.49], G:[ACC: %25.39, LOSS: 237.32]
E: 135, D:[ACC: %100.00, LOSS: 152.77], G:[ACC: %21.09, LOSS: 248.19]
E: 136, D:[ACC: %100.00, LOSS: 148.

E: 240, D:[ACC: %100.00, LOSS: 115.15], G:[ACC: %99.61, LOSS: 133.61]
E: 241, D:[ACC: %100.00, LOSS: 111.49], G:[ACC: %99.61, LOSS: 131.48]
E: 242, D:[ACC: %100.00, LOSS: 111.07], G:[ACC: %98.83, LOSS: 120.00]
E: 243, D:[ACC: %100.00, LOSS: 106.17], G:[ACC: %100.00, LOSS: 122.74]
E: 244, D:[ACC: %100.00, LOSS: 110.59], G:[ACC: %98.44, LOSS: 133.27]
E: 245, D:[ACC: %99.80, LOSS: 114.58], G:[ACC: %99.61, LOSS: 123.83]
E: 246, D:[ACC: %100.00, LOSS: 111.88], G:[ACC: %98.83, LOSS: 126.66]
E: 247, D:[ACC: %100.00, LOSS: 108.28], G:[ACC: %98.44, LOSS: 129.74]
E: 248, D:[ACC: %100.00, LOSS: 113.63], G:[ACC: %98.83, LOSS: 123.43]
E: 249, D:[ACC: %99.80, LOSS: 106.05], G:[ACC: %98.44, LOSS: 128.86]
E: 250, D:[ACC: %100.00, LOSS: 105.63], G:[ACC: %99.22, LOSS: 124.37]
E: 251, D:[ACC: %100.00, LOSS: 110.81], G:[ACC: %98.83, LOSS: 123.47]
E: 252, D:[ACC: %100.00, LOSS: 111.87], G:[ACC: %99.22, LOSS: 121.73]
E: 253, D:[ACC: %100.00, LOSS: 108.78], G:[ACC: %96.88, LOSS: 127.90]
E: 254, D:[ACC: %100.

E: 358, D:[ACC: %100.00, LOSS: 86.02], G:[ACC: %100.00, LOSS: 82.25]
E: 359, D:[ACC: %100.00, LOSS: 88.66], G:[ACC: %100.00, LOSS: 86.24]
E: 360, D:[ACC: %100.00, LOSS: 85.96], G:[ACC: %100.00, LOSS: 89.68]
E: 361, D:[ACC: %100.00, LOSS: 88.36], G:[ACC: %100.00, LOSS: 84.68]
E: 362, D:[ACC: %100.00, LOSS: 83.11], G:[ACC: %100.00, LOSS: 84.76]
E: 363, D:[ACC: %100.00, LOSS: 86.38], G:[ACC: %100.00, LOSS: 90.21]
E: 364, D:[ACC: %100.00, LOSS: 83.96], G:[ACC: %100.00, LOSS: 86.27]
E: 365, D:[ACC: %100.00, LOSS: 87.17], G:[ACC: %100.00, LOSS: 85.47]
E: 366, D:[ACC: %100.00, LOSS: 87.54], G:[ACC: %100.00, LOSS: 87.16]
E: 367, D:[ACC: %100.00, LOSS: 91.04], G:[ACC: %100.00, LOSS: 90.19]
E: 368, D:[ACC: %100.00, LOSS: 86.83], G:[ACC: %100.00, LOSS: 87.36]
E: 369, D:[ACC: %100.00, LOSS: 87.53], G:[ACC: %100.00, LOSS: 95.16]
E: 370, D:[ACC: %100.00, LOSS: 87.78], G:[ACC: %100.00, LOSS: 93.21]
E: 371, D:[ACC: %100.00, LOSS: 87.51], G:[ACC: %100.00, LOSS: 82.46]
E: 372, D:[ACC: %99.80, LOSS: 90.3

...

E: 835, D:[ACC: %100.00, LOSS: 45.09], G:[ACC: %2.73, LOSS: 333.81]
E: 836, D:[ACC: %100.00, LOSS: 38.84], G:[ACC: %24.61, LOSS: 194.61]
E: 837, D:[ACC: %100.00, LOSS: 40.92], G:[ACC: %62.89, LOSS: 113.73]
E: 838, D:[ACC: %100.00, LOSS: 40.63], G:[ACC: %80.86, LOSS: 86.19]
E: 839, D:[ACC: %100.00, LOSS: 39.46], G:[ACC: %90.62, LOSS: 69.88]
E: 840, D:[ACC: %100.00, LOSS: 37.91], G:[ACC: %96.88, LOSS: 61.89]
E: 841, D:[ACC: %100.00, LOSS: 39.66], G:[ACC: %99.61, LOSS: 54.61]
E: 842, D:[ACC: %100.00, LOSS: 38.32], G:[ACC: %97.66, LOSS: 56.95]
E: 843, D:[ACC: %100.00, LOSS: 38.47], G:[ACC: %98.83, LOSS: 54.72]
E: 844, D:[ACC: %100.00, LOSS: 39.13], G:[ACC: %100.00, LOSS: 50.90]
E: 845, D:[ACC: %100.00, LOSS: 36.87], G:[ACC: %97.27, LOSS: 55.21]
E: 846, D:[ACC: %100.00, LOSS: 37.29], G:[ACC: %99.22, LOSS: 51.58]
E: 847, D:[ACC: %100.00, LOSS: 39.75], G:[ACC: %99.61, LOSS: 46.05]
E: 848, D:[ACC: %100.00, LOSS: 39.31], G:[ACC: %98.83, LOSS: 48.00]
E: 849, D:[ACC: %100.00, LOSS: 37.44], G:[ACC

E: 955, D:[ACC: %99.80, LOSS: 29.78], G:[ACC: %100.00, LOSS: 39.11]
E: 956, D:[ACC: %100.00, LOSS: 30.26], G:[ACC: %100.00, LOSS: 38.25]
E: 957, D:[ACC: %100.00, LOSS: 30.20], G:[ACC: %100.00, LOSS: 38.57]
E: 958, D:[ACC: %100.00, LOSS: 31.62], G:[ACC: %100.00, LOSS: 39.03]
E: 959, D:[ACC: %100.00, LOSS: 29.11], G:[ACC: %100.00, LOSS: 39.10]
E: 960, D:[ACC: %100.00, LOSS: 28.92], G:[ACC: %100.00, LOSS: 38.47]
E: 961, D:[ACC: %100.00, LOSS: 29.48], G:[ACC: %100.00, LOSS: 40.80]
E: 962, D:[ACC: %100.00, LOSS: 29.36], G:[ACC: %100.00, LOSS: 39.15]
E: 963, D:[ACC: %100.00, LOSS: 30.42], G:[ACC: %100.00, LOSS: 37.99]
E: 964, D:[ACC: %100.00, LOSS: 29.35], G:[ACC: %100.00, LOSS: 36.34]
E: 965, D:[ACC: %100.00, LOSS: 30.74], G:[ACC: %100.00, LOSS: 36.52]
E: 966, D:[ACC: %100.00, LOSS: 29.05], G:[ACC: %100.00, LOSS: 37.97]
E: 967, D:[ACC: %100.00, LOSS: 30.80], G:[ACC: %100.00, LOSS: 38.38]
E: 968, D:[ACC: %100.00, LOSS: 29.14], G:[ACC: %100.00, LOSS: 36.68]
E: 969, D:[ACC: %100.00, LOSS: 29.7

E: 1073, D:[ACC: %100.00, LOSS: 23.16], G:[ACC: %100.00, LOSS: 26.04]
E: 1074, D:[ACC: %100.00, LOSS: 24.46], G:[ACC: %100.00, LOSS: 25.76]
E: 1075, D:[ACC: %100.00, LOSS: 24.54], G:[ACC: %100.00, LOSS: 25.83]
E: 1076, D:[ACC: %100.00, LOSS: 23.56], G:[ACC: %100.00, LOSS: 24.50]
E: 1077, D:[ACC: %100.00, LOSS: 24.07], G:[ACC: %100.00, LOSS: 26.21]
E: 1078, D:[ACC: %100.00, LOSS: 25.07], G:[ACC: %100.00, LOSS: 26.21]
E: 1079, D:[ACC: %100.00, LOSS: 24.69], G:[ACC: %100.00, LOSS: 27.35]
E: 1080, D:[ACC: %100.00, LOSS: 23.67], G:[ACC: %100.00, LOSS: 27.89]
E: 1081, D:[ACC: %100.00, LOSS: 26.17], G:[ACC: %100.00, LOSS: 26.04]
E: 1082, D:[ACC: %100.00, LOSS: 23.83], G:[ACC: %100.00, LOSS: 28.88]
E: 1083, D:[ACC: %100.00, LOSS: 23.73], G:[ACC: %100.00, LOSS: 27.82]
E: 1084, D:[ACC: %100.00, LOSS: 24.78], G:[ACC: %100.00, LOSS: 27.70]
E: 1085, D:[ACC: %100.00, LOSS: 24.91], G:[ACC: %100.00, LOSS: 27.22]
E: 1086, D:[ACC: %100.00, LOSS: 22.83], G:[ACC: %100.00, LOSS: 27.24]
E: 1087, D:[ACC: %10

E: 1191, D:[ACC: %100.00, LOSS: 19.60], G:[ACC: %99.61, LOSS: 34.00]
E: 1192, D:[ACC: %100.00, LOSS: 19.85], G:[ACC: %100.00, LOSS: 31.18]
E: 1193, D:[ACC: %100.00, LOSS: 20.04], G:[ACC: %100.00, LOSS: 29.99]
E: 1194, D:[ACC: %100.00, LOSS: 21.63], G:[ACC: %100.00, LOSS: 27.33]
E: 1195, D:[ACC: %100.00, LOSS: 19.52], G:[ACC: %100.00, LOSS: 28.58]
E: 1196, D:[ACC: %100.00, LOSS: 21.79], G:[ACC: %100.00, LOSS: 28.38]
E: 1197, D:[ACC: %100.00, LOSS: 21.29], G:[ACC: %100.00, LOSS: 27.78]
E: 1198, D:[ACC: %100.00, LOSS: 19.94], G:[ACC: %100.00, LOSS: 29.02]
E: 1199, D:[ACC: %100.00, LOSS: 19.23], G:[ACC: %100.00, LOSS: 27.72]
E: 1200, D:[ACC: %100.00, LOSS: 19.82], G:[ACC: %99.61, LOSS: 30.90]
E: 1201, D:[ACC: %100.00, LOSS: 18.64], G:[ACC: %99.61, LOSS: 31.09]
E: 1202, D:[ACC: %100.00, LOSS: 20.78], G:[ACC: %100.00, LOSS: 28.72]
E: 1203, D:[ACC: %100.00, LOSS: 20.89], G:[ACC: %100.00, LOSS: 29.64]
E: 1204, D:[ACC: %100.00, LOSS: 20.39], G:[ACC: %100.00, LOSS: 31.88]
E: 1205, D:[ACC: %100.0

E: 1310, D:[ACC: %100.00, LOSS: 17.80], G:[ACC: %99.61, LOSS: 31.29]
E: 1311, D:[ACC: %100.00, LOSS: 16.85], G:[ACC: %99.22, LOSS: 32.62]
E: 1312, D:[ACC: %100.00, LOSS: 17.34], G:[ACC: %98.44, LOSS: 31.17]
E: 1313, D:[ACC: %99.80, LOSS: 18.85], G:[ACC: %98.83, LOSS: 32.41]
E: 1314, D:[ACC: %100.00, LOSS: 17.41], G:[ACC: %99.22, LOSS: 31.36]
E: 1315, D:[ACC: %100.00, LOSS: 16.84], G:[ACC: %99.22, LOSS: 30.32]
E: 1316, D:[ACC: %100.00, LOSS: 17.57], G:[ACC: %100.00, LOSS: 30.18]
E: 1317, D:[ACC: %100.00, LOSS: 17.43], G:[ACC: %99.22, LOSS: 31.71]
E: 1318, D:[ACC: %100.00, LOSS: 16.34], G:[ACC: %99.61, LOSS: 30.64]
E: 1319, D:[ACC: %100.00, LOSS: 17.28], G:[ACC: %99.61, LOSS: 31.80]
E: 1320, D:[ACC: %100.00, LOSS: 16.79], G:[ACC: %99.22, LOSS: 31.74]
E: 1321, D:[ACC: %100.00, LOSS: 16.47], G:[ACC: %98.83, LOSS: 33.06]
E: 1322, D:[ACC: %100.00, LOSS: 16.16], G:[ACC: %98.44, LOSS: 32.43]
E: 1323, D:[ACC: %100.00, LOSS: 16.08], G:[ACC: %98.83, LOSS: 31.37]
E: 1324, D:[ACC: %100.00, LOSS: 17

## Plot Loss

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();

In [ ]:
generator.save('./gen.h5')

In [ ]:
discriminator.save('./disc.h5')

In [ ]:
gan.save('./gan.h5')

In [ ]:
gan.metrics_names

In [ ]:
## visualize results
synthesized = generator.predict([conditions, noise])
visualizeGAN(8000, None, synthesized, conditions)